# Intro to gragh convolutional surrogate models
Eamon Whalen

In [1]:
import sys
import numpy as np
import pandas as pd
import altair as alt

sys.path.append('./models')
from feastnetSurrogateModel import FeaStNet

sys.path.append('./readers')
from loadGhGraphs import loadGhGraphs

sys.path.append('./visualization')
from altTrussViz import plotTruss, interactiveErrorPlot

sys.path.append('./util')
from gcnSurrogateUtil import *

## 1. Load simulation data

In [2]:
doeFile = "/home/ewhalen/projects/data/trusses/2D_Truss_v1.3/design_7_N_1000.csv"
allGraphsUnfiltered = loadGhGraphs(doeFile, NUM_DV=5)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphsUnfiltered]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,1000.000000
mean,0.199516
std,2.524026
min,0.006570
25%,0.015425
50%,0.024334
75%,0.046981
max,67.325867


In [3]:
plotTruss(allGraphsUnfiltered[0], showDeformed=True, defScale=100)

alt.LayerChart(...)

## 2. Filter and partition

In [4]:
allGraphs = filterbyDisp(allGraphsUnfiltered, 0.9)
trainData, valData, testData = partitionGraphList(allGraphs)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphs]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,900.000000
mean,0.028952
std,0.019071
min,0.006570
25%,0.014951
50%,0.021719
75%,0.038367
max,0.097861


## 3. Train a GCN

In [5]:
gcn = FeaStNet()
history = gcn.trainModel(trainData, valData, 
                     epochs=100, 
                     batch_size=256, 
                     flatten=True, 
                     logTrans=False, 
                     ssTrans=True, 
                     saveDir='./results/gcn01/')

plotHistory(history)

epoch: 0   trainLoss: 9.8400e-01   valLoss:9.5580e-01  time: 1.39e+00
epoch: 1   trainLoss: 8.7333e-01   valLoss:9.5192e-01  time: 1.07e+00
epoch: 2   trainLoss: 8.2259e-01   valLoss:9.6617e-01  time: 1.05e+00
epoch: 3   trainLoss: 8.2614e-01   valLoss:1.0219e+00  time: 1.03e+00
epoch: 4   trainLoss: 7.9439e-01   valLoss:1.0163e+00  time: 1.05e+00
epoch: 5   trainLoss: 7.5646e-01   valLoss:9.7160e-01  time: 1.05e+00
epoch: 6   trainLoss: 7.5111e-01   valLoss:9.2382e-01  time: 1.04e+00
epoch: 7   trainLoss: 6.7522e-01   valLoss:8.7907e-01  time: 1.06e+00
epoch: 8   trainLoss: 6.7062e-01   valLoss:7.9299e-01  time: 1.06e+00
epoch: 9   trainLoss: 6.8911e-01   valLoss:7.0669e-01  time: 1.08e+00
epoch: 10   trainLoss: 6.4982e-01   valLoss:6.6469e-01  time: 1.05e+00
epoch: 11   trainLoss: 6.0214e-01   valLoss:6.3540e-01  time: 1.05e+00
epoch: 12   trainLoss: 5.8906e-01   valLoss:6.1675e-01  time: 1.04e+00
epoch: 13   trainLoss: 5.7140e-01   valLoss:5.9606e-01  time: 1.05e+00
epoch: 14   trai

alt.Chart(...)

## 4. Test the GCN

In [6]:
trainRes = gcn.testModel(trainData)
testRes = gcn.testModel(testData)
pd.DataFrame([trainRes, testRes], index=['train', 'test'])

,mse,mae,mre,peakR2,maxAggR2,meanAggR2,minAggR2
train,0.000019,0.002954,0.131593,0.858602,0.947091,0.382422,-8.508078
test,0.000027,0.003258,0.147443,0.725597,0.876594,0.332282,-7.850835


## 5. Visualize some predictions

In [7]:
i = 9
pred = gcn.predict([testData[i]])[0]
plotTruss(testData[i], showDeformed=True, defScale=100, prediction=pred)

alt.LayerChart(...)

In [8]:
alt.data_transformers.enable('json')
allPreds = gcn.predict(testData)
interactiveErrorPlot(testData, allPreds)

alt.HConcatChart(...)